In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit

spark = SparkSession.builder.appName("DeltaTimeTravel").getOrCreate()

data = [
    (101, "Dhoni hits a six"),
    (102, "Rohit scores a four"),
    (103, "Kohli gets bowled")
]
columns = ["match_id", "commentary"]

commentary_df = spark.createDataFrame(data, columns)
commentary_df.show()


+--------+-------------------+
|match_id|         commentary|
+--------+-------------------+
|     101|   Dhoni hits a six|
|     102|Rohit scores a four|
|     103|  Kohli gets bowled|
+--------+-------------------+



In [0]:

commentary_df.write.format("delta").mode("overwrite").save("/tmp/commentary_delta")


In [0]:
%sql

CREATE TABLE commentary_delta
USING DELTA
LOCATION '/tmp/commentary_delta';


In [0]:
updated_df = commentary_df.withColumn("analyst_remarks", lit("Reviewed"))

updated_df.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .save("/tmp/commentary_delta")



In [0]:
%sql

DESCRIBE HISTORY commentary_delta;


version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
1,2025-04-10T14:44:25.000+0000,7519223837836881,dakshinishanmugam7@gmail.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(1543009371561748),0410-142623-ky1opn6m,0,WriteSerializable,false,"Map(numFiles -> 3, numOutputRows -> 3, numOutputBytes -> 3772)",null,Databricks-Runtime/12.2.x-scala2.12
0,2025-04-10T14:41:59.000+0000,7519223837836881,dakshinishanmugam7@gmail.com,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(1543009371561748),0410-142623-ky1opn6m,null,WriteSerializable,false,"Map(numFiles -> 3, numOutputRows -> 3, numOutputBytes -> 2890)",null,Databricks-Runtime/12.2.x-scala2.12


In [0]:
%sql

SELECT * FROM delta.`/tmp/commentary_delta` VERSION AS OF 0;

SELECT * FROM delta.`/tmp/commentary_delta` VERSION AS OF 1;


match_id,commentary,analyst_remarks
102,Rohit scores a four,Reviewed
103,Kohli gets bowled,Reviewed
101,Dhoni hits a six,Reviewed
